**Game:** *Alien Rescue Mission*

**Objective:**

Rescue a stranded astronaut on an alien planet using the shortest and safest path while avoiding hostile creatures. The player uses the A* search algorithm to navigate a hazardous terrain, considering both distance and danger. Each step is an Action, and each position on the planet is a State. The goal is to rescue the astronaut and return to the spaceship as quickly as possible.

**1. Setup of the Game**

Planet Terrain (Problem Space)
The terrain is represented as a grid (4x4 or larger), where each cell is a State. Some cells contain dangerous obstacles or hostile creatures, while others are clear paths. The player starts at the spaceship (start state), and the astronaut is stranded somewhere on the grid (goal state). After rescuing the astronaut, the player must return to the spaceship.

**Example of 4x4 gride**




In [5]:
[Spaceship] [   ] [   ] [Alien Nest]
[   ]       [ X ] [   ] [   ]
[Rock]      [   ] [Alien Nest] [   ]
[   ]       [   ] [   ] [Astronaut]


SyntaxError: invalid syntax (<ipython-input-5-4b4e67644e3f>, line 1)

**Key:**

Empty cells: Safe path.

"X": Impassable obstacle (e.g., rock).

"Alien Nest": Dangerous area that adds cost (danger level) to pass.
Spaceship: Starting point.

Astronaut: Goal.

**2. Game Rules**

**States:**

Each cell in the grid represents a state. The starting state is the player's position at the spaceship, and the goal is to reach the astronaut and bring them back. The A* algorithm uses a combination of distance and danger to find the most optimal path.

**Actions:**

The player can move up, down, left, or right. Moving into a neighboring state is considered an action. Movement through dangerous zones (like alien nests) adds a penalty, making it a less desirable route.

**Goals:**

**Primary Goal:** Rescue the astronaut.

**Secondary Goal:** Return to the spaceship after rescue with minimal danger and the shortest distance.

**3. A* Algorithm**

The A* algorithm uses two factors to calculate the best path:

* g(n): The actual cost from the start to the current state (distance
traveled and danger penalties).
* h(n): The heuristic estimate of the cost from the current state to the goal (straight-line distance to the astronaut or spaceship).
*f(n) = g(n) + h(n): The total cost of a path, which the A* algorithm uses to decide the next move.

In this game, the algorithm needs to account for both distance and the "danger cost" of moving through hostile areas. A player can pass through alien nests, but it adds to the overall cost.

**4. Game Process**

Start at the spaceship (initial state).

Begin searching for the astronaut using the A* algorithm, balancing between the shortest path and avoiding hazardous zones.
Move through the grid by selecting neighboring states with the lowest total cost (f(n)).

If a neighboring state has an obstacle (rock), you cannot pass through it.
Alien nests are passable but increase the g(n) cost due to their danger.
Rescue the astronaut (goal state).

After reaching the astronaut, return to the spaceship using the same A* algorithm to find the safest and shortest route back.
Game ends when the player safely returns to the spaceship with the astronaut.

**5. Challenges:**

Hostile Zones: Alien nests may block the most direct route, forcing the player to choose between a longer but safer path or risking danger to take the shorter route.
Impassable Obstacles: Some areas may be completely blocked, requiring clever navigation around them.

**6. Game Play Example **

 The player starts at the spaceship (0,0).

**First Move:** A* calculates that moving right to (0,1) is safer than moving down toward the alien nest.

**Next Moves:** The player continues navigating through safe zones, avoiding alien nests, until reaching the astronaut at (3,3).

**Return Journey:** After rescuing the astronaut, the player uses the A* algorithm again to find the safest path back, this time with more emphasis on avoiding danger than simply minimizing distance.


In [ ]:
import heapq  # For priority queue in A* algorithm
import random  # For placing obstacles and astronaut randomly

# Function to create a grid with the spaceship at (0,0) and astronaut at a random position
def create_grid(size):
    grid = [[' ' for _ in range(size)] for _ in range(size)]  # Create an empty grid
    grid[0][0] = 'S'  # 'S' marks the starting point (spaceship)

    # Place the astronaut ('A') randomly, ensuring it's not at the start
    astronaut_x, astronaut_y = random.randint(0, size-1), random.randint(0, size-1)
    while (astronaut_x, astronaut_y) == (0, 0):
        astronaut_x, astronaut_y = random.randint(0, size-1), random.randint(0, size-1)

    grid[astronaut_x][astronaut_y] = 'A'  # Place the astronaut
    return grid, (astronaut_x, astronaut_y)

# Function to add obstacles (like alien nests) to the grid
def add_obstacles(grid, num_obstacles):
    size = len(grid)
    for _ in range(num_obstacles):
        obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        # Ensure obstacles are not placed on 'S' (spaceship) or 'A' (astronaut)
        while grid[obstacle_x][obstacle_y] in ['S', 'A']:
            obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        grid[obstacle_x][obstacle_y] = 'X'  # Place the obstacle ('X')
    return grid

# Function to check if a position is valid
def is_valid_position(grid, x, y):
    size = len(grid)
    return 0 <= x < size and 0 <= y < size and grid[x][y] != 'X'  # Ensure the position is not blocked

# Heuristic function: Manhattan distance between the current state and the goal
def heuristic(current, goal):
    return abs(current[0] - goal[0]) + abs(current[1] - goal[1])

# A* search algorithm to find the optimal path considering both distance and cost (danger)
def astar(grid, start, goal):
    size = len(grid)
    open_list = []
    heapq.heappush(open_list, (0, start))  # Priority queue with (cost, state)
    g_score = {start: 0}  # Actual cost to reach each state
    parent = {}  # To reconstruct the path
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Possible moves: up, down, left, right

    while open_list:
        current_cost, current = heapq.heappop(open_list)  # Get the state with the lowest cost
        if current == goal:
            break  # If we reached the goal (astronaut)

        for direction in directions:
            next_x = current[0] + direction[0]
            next_y = current[1] + direction[1]
            next_state = (next_x, next_y)

            if is_valid_position(grid, next_x, next_y):
                danger_cost = 2 if grid[next_x][next_y] == 'D' else 1  # Add danger cost if moving through dangerous zone
                new_g_score = g_score[current] + danger_cost

                if next_state not in g_score or new_g_score < g_score[next_state]:
                    g_score[next_state] = new_g_score
                    f_score = new_g_score + heuristic(next_state, goal)
                    heapq.heappush(open_list, (f_score, next_state))
                    parent[next_state] = current

    # Reconstruct the path
    path = []
    current = goal
    while current in parent:
        path.append(current)
        current = parent[current]
    path.append(start)
    path.reverse()

    return path

# Function to print the grid with the path marked
def print_grid_with_path(grid, path):
    grid_with_path = [row.copy() for row in grid]  # Copy the original grid
    for (x, y) in path:
        if grid_with_path[x][y] not in ['S', 'A']:  # Don't overwrite start 'S' or astronaut 'A'
            grid_with_path[x][y] = '*'

    print("\nGrid with Path:")
    print('-' * (len(grid_with_path) * 4 + 1))
    for row in grid_with_path:
        print('| ' + ' | '.join(row) + ' |')
        print('-' * (len(grid_with_path) * 4 + 1))

# Main function to play the game
def alien_rescue_mission():
    # Ask user for grid size and number of obstacles
    size = int(input("Enter the grid size (e.g., 6 for a 6x6 grid): "))
    num_obstacles = int(input(f"Enter the number of obstacles (less than {size * size - 2}): "))

    # Create the grid and place the astronaut randomly
    grid, goal = create_grid(size)

    # Add random obstacles to the grid
    grid = add_obstacles(grid, num_obstacles)

    # Add some dangerous zones ('D') for extra cost
    num_danger_zones = int(input(f"Enter the number of dangerous zones (less than {size * size - num_obstacles - 2}): "))
    for _ in range(num_danger_zones):
        danger_x, danger_y = random.randint(0, size-1), random.randint(0, size-1)
        while grid[danger_x][danger_y] in ['S', 'A', 'X']:
            danger_x, danger_y = random.randint(0, size-1), random.randint(0, size-1)
        grid[danger_x][danger_y] = 'D'

    # Print the initial grid
    print("\nInitial Grid:")
    print_grid_with_path(grid, [])

    # Start the search for the astronaut
    print("\nSearching for the astronaut using A* algorithm...\n")
    path = astar(grid, (0, 0), goal)

    # Print the final grid with the path marked
    print("\nPath to the Astronaut:")
    print_grid_with_path(grid, path)

# Run the game
alien_rescue_mission()


Enter the grid size (e.g., 6 for a 6x6 grid): 5
Enter the number of obstacles (less than 23): 20
Enter the number of dangerous zones (less than 3): 2

Initial Grid:

Grid with Path:
---------------------
| S | X | X | X | X |
---------------------
| X | X | D |   | X |
---------------------
| X |   | D |   |   |
---------------------
| X | X | A | X |   |
---------------------
| X |   | X |   | X |
---------------------

Searching for the astronaut using A* algorithm...


Path to the Astronaut:

Grid with Path:
---------------------
| S | X | X | X | X |
---------------------
| X | X | D |   | X |
---------------------
| X |   | D |   |   |
---------------------
| X | X | A | X |   |
---------------------
| X |   | X |   | X |
---------------------


**Key Changes:**

**A* Algorithm:**  
Used a priority queue (heapq) to implement A*, which balances between the shortest path and danger cost (dangerous zones marked as 'D' add extra movement cost).

**Grid Elements:**

* S: Starting point (spaceship).
* A: Goal (astronaut).
* X: Obstacles (alien nests).
* D: Dangerous zones that add a penalty to the path cost.

Dynamic Grid: The grid is dynamically generated with user input for size, obstacles, and danger zones.